# ML testing: experiment #3e- harmonized with neuroHarmonize (neurocombat-GAM)

This notebook involves testing for the clinical harmonization paper. This notebook shows mixed_dataset (TOP + StrokeMRI) based models with neuroHarmonize harmonization to Insight 46 and Sabre datasets (the strokeMRI and TOP become one dataset)

Data: StrokeMRI, TOP, SABRE, Insight46, EDIS, (HELIUS pending)

Harmonisation: neuroHarmonize

Training data: NORMENT whichh is StrokeMRI and TOP togehter

Testing data: test set from NORMENT

Futher data applied to: SABRE, Insight46, EDIS, (HELIUS pending)

Validation method: K-fold, double-stratified

Brain-age algorithms: LR, lasso, extra trees, elasticCV net fully tested (but not optimized parameters), additionals partly

Outputs: SubjectID, real age, predicted age of validation and testing sets

### import libraries

In [1]:
import os       # using operating system dependent functionality (folders)
import sys

import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

# # demo stuff
# import ipywidgets as widgets
# import seaborn 

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score

import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

### import data

In [2]:
filepath_mri_for_ids = '../open_work/internal_results/cleaned_pvc2s' 
filename_mri_for_ids = os.path.join(filepath_mri_for_ids,'StrokeMRI_pvc2c.csv') 

filepath_top_for_ids = '../open_work/internal_results/cleaned_pvc2s/' 
filename_top_for_ids = os.path.join(filepath_top_for_ids,'TOP_pvc2c.csv') 



filepath_topmri = 'harmonizations/harm_results/neurocharm/' 
filename_topmri = os.path.join(filepath_topmri,'topmri_vs_sabre_5way.csv') 

filepath_sabre = 'harmonizations/harm_results/neurocharm/'
filename_sabre = os.path.join(filepath_topmri,'sabre_vs_topmri_5way.csv') 

filepath_insight46 = 'harmonizations/harm_results/neurocharm/'
filename_insight46 =  os.path.join(filepath_topmri,'insight_vs_topmri_5way.csv') 

filepath_edis = filepath_sabre
filename_edis =  os.path.join(filepath_sabre,'edis_vs_topmri_5way.csv') 
# 
filepath_helius = filepath_sabre
filename_helius =  os.path.join(filepath_sabre,'helius_vs_topmri_5way.csv') 



# read in data

TOPMRI = pd.read_csv(filename_topmri)
SABRE = pd.read_csv(filename_sabre)
Insight46 = pd.read_csv(filename_insight46)
# take extra column off
TOPMRI = TOPMRI.drop(TOPMRI.columns[0],axis=1)
SABRE = SABRE.drop(SABRE.columns[0],axis=1)
#StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)
Insight46 = Insight46.drop(Insight46.columns[0],axis=1)
IDS_TOP =  pd.read_csv(filename_top_for_ids)
IDS_MRI =  pd.read_csv(filename_mri_for_ids)

In [18]:
TOPMRI.columns

Index(['participant_id', 'age', 'sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf'],
      dtype='object')

In [19]:
#SABRE

In [20]:
# # Now we need to flip the sex back to numbers for a correlation
# sex_mapping = {'F':0,'M':1}
# TOPMRI = TOPMRI.assign(sex = TOPMRI.sex.map(sex_mapping))
# TOPMRI.head(3)

In [21]:
# Insight46 = Insight46.assign(sex = Insight46.sex.map(sex_mapping))
# #Insight46.head(3)

In [22]:
coly = TOPMRI.columns
#SABRE = SABRE.assign(sex = SABRE.sex.map(sex_mapping))
SABRE = SABRE[coly]
SABRE.head(3)

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf
0,sub-301662_1,79.0,0,0.586358,0.448759,0.458928,0.389714,0.689305,0.025156,29.638872,1.755722,0.630854,0.595168,0.707843,0.612607,18.014589,70.466012,61.450818,44.958262,56.605375
1,sub-600024_1,70.0,1,0.538437,0.490314,0.370691,0.384937,0.735037,0.026859,34.128897,1.427242,0.348795,0.399334,0.442168,0.427039,14.170078,57.896764,54.618764,41.683278,47.354919
2,sub-600134_1,62.0,0,0.684834,0.652095,0.470113,0.377174,0.732950,0.061435,84.191774,-0.970254,0.355570,0.405248,0.375765,0.432835,34.927001,115.560714,101.682269,81.192884,88.804595


In [23]:
TOPMRI.shape

(1107, 20)

In [24]:
SABRE.shape

(706, 20)

In [25]:
SABRE =SABRE.dropna()
#SABRE.isna().sum()

# Now we need to break up the TOP and MRI datasets as well, and format them like the others

In [26]:
set_top_ids = set(IDS_TOP.participant_id)
set_mri_ids = set(IDS_MRI.participant_id)
StrokeMRI = TOPMRI[TOPMRI['participant_id'].isin(list(set_mri_ids))]
TOP = TOPMRI[TOPMRI['participant_id'].isin(list(set_top_ids))]
TOP.head(3)

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf
0,sub-0668_1,50.40,1,0.617501,0.470392,0.236052,0.463182,0.819954,-0.005404,12.732741,2.821634,0.248649,0.558961,0.82831,0.719521,18.921497,87.011810,80.241341,55.807275,70.977497
1,sub-0532_1,37.02,1,0.693920,0.579498,0.294685,0.439796,0.811614,-0.006151,19.612609,2.001670,0.337168,0.862546,0.60858,0.716976,18.218930,98.415147,90.024820,66.535156,79.219991
2,sub-0529_1,30.57,1,0.641051,0.444492,0.265101,0.473141,0.796567,-0.002403,15.470687,1.955187,0.618026,0.620556,0.53582,0.596405,20.082497,106.272833,99.973628,71.505620,86.542509


In [27]:
# StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
# StrokeMRI.head(3)

In [28]:
StrokeMRI

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf
529,sub-5917601_1,70.713706,1,0.620299,0.426864,0.434966,0.421630,0.706982,0.013864,13.227193,4.364039,1.068751,0.364302,1.221599,0.866289,36.235943,84.475644,76.876389,60.466633,66.605412
530,sub-5931802_1,47.583562,1,0.616693,0.484783,0.295519,0.439477,0.786037,-0.002487,22.553396,2.280488,0.136074,0.445344,0.384868,0.406642,22.292957,89.518129,82.482392,67.060451,73.466224
531,sub-5911901_1,74.104110,1,0.542521,0.474333,0.434024,0.374280,0.701546,0.038571,38.018110,3.170695,0.107762,0.628749,0.930111,0.450528,23.797849,71.339877,61.756876,58.859106,59.570497
532,sub-5910201_1,68.843836,0,0.612575,0.480048,0.416291,0.406692,0.725352,0.017859,21.703612,0.067288,0.003601,0.011399,0.335796,0.065553,18.416714,68.163076,62.343797,47.567858,55.030655
533,sub-5928701_1,61.978082,1,0.608974,0.510900,0.463166,0.385488,0.707255,-0.001957,14.562046,3.747040,0.504116,0.700305,0.572789,0.778108,23.602228,107.798177,96.881824,74.677729,82.591893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,sub-5928501_1,67.178082,1,0.545356,0.434526,0.299969,0.424104,0.764033,0.003447,25.232915,3.558449,0.167488,0.397538,0.654104,0.681550,18.581290,67.255671,58.751153,39.633615,48.245258
1103,sub-5910502_1,67.794521,1,0.539692,0.430524,0.317793,0.417475,0.751390,-0.004552,27.379342,3.296029,0.299296,0.595163,0.726507,0.616639,16.811391,64.346135,58.592201,50.353841,52.097482
1104,sub-5931002_1,54.769863,0,0.644108,0.546530,0.392740,0.405719,0.752413,0.005652,36.178471,-0.608323,1.199931,0.587832,0.569897,0.627489,17.788713,64.184644,56.535437,43.445700,51.473135
1105,sub-5925701_1,66.639344,1,0.559078,0.461613,0.321087,0.415037,0.760508,0.000712,25.248501,3.835611,0.118999,1.158012,1.131658,1.050815,17.138859,53.898865,56.003862,36.754183,43.953223


In [29]:
#TOP = TOP.assign(sex = TOP.sex.map(sex_mapping))
TOP.head(3)

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf
0,sub-0668_1,50.40,1,0.617501,0.470392,0.236052,0.463182,0.819954,-0.005404,12.732741,2.821634,0.248649,0.558961,0.82831,0.719521,18.921497,87.011810,80.241341,55.807275,70.977497
1,sub-0532_1,37.02,1,0.693920,0.579498,0.294685,0.439796,0.811614,-0.006151,19.612609,2.001670,0.337168,0.862546,0.60858,0.716976,18.218930,98.415147,90.024820,66.535156,79.219991
2,sub-0529_1,30.57,1,0.641051,0.444492,0.265101,0.473141,0.796567,-0.002403,15.470687,1.955187,0.618026,0.620556,0.53582,0.596405,20.082497,106.272833,99.973628,71.505620,86.542509


In [30]:
# check for any duplicated patients between stroke and mri
strokers = set(StrokeMRI.participant_id)
topers = set(TOP.participant_id)
z = strokers.intersection(topers)
print(z) 

set()


In [31]:
# make mixed StrokeMRI and TOP dataset
#mixed_data = pd.concat([TOP, StrokeMRI], sort=False)
mixed_data = TOPMRI

In [32]:
mixed_data.shape

(1107, 20)

In [33]:
mixed_data.head()

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf
0,sub-0668_1,50.40,1,0.617501,0.470392,0.236052,0.463182,0.819954,-0.005404,12.732741,2.821634,0.248649,0.558961,0.828310,0.719521,18.921497,87.011810,80.241341,55.807275,70.977497
1,sub-0532_1,37.02,1,0.693920,0.579498,0.294685,0.439796,0.811614,-0.006151,19.612609,2.001670,0.337168,0.862546,0.608580,0.716976,18.218930,98.415147,90.024820,66.535156,79.219991
2,sub-0529_1,30.57,1,0.641051,0.444492,0.265101,0.473141,0.796567,-0.002403,15.470687,1.955187,0.618026,0.620556,0.535820,0.596405,20.082497,106.272833,99.973628,71.505620,86.542509
3,sub-0393_1,47.05,1,0.628643,0.489904,0.189978,0.475536,0.853308,-0.002670,22.568832,2.521627,0.353867,0.746916,0.518321,0.738841,20.834204,89.468453,80.379688,59.770654,72.604385
4,sub-0593_1,44.63,1,0.636804,0.473800,0.252016,0.464884,0.812493,-0.003348,31.295887,2.085748,0.435600,0.427990,0.504488,0.398955,18.066908,79.233881,76.460555,66.449866,69.887736


In [35]:
output_folder = '3_NeuroHarmonize'

os.makedirs(output_folder, exist_ok=True)

## Build ML models

In [36]:
ml_matrix = mixed_data.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
y = ml_matrix['age'].values
y=y.astype('float')

In [37]:
linr_k_frame, linr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('linear regression', 'req_neuroharm_mix_linr', LinearRegression(), ml_matrix, X[:,1:], y)

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

In [38]:
linr_k_frame

,algorithm,fold,file_name,mae,r2,explained_variance
0,linear regression-0,0,req_neuroharm_mix_linr.0,4.720398,0.846156,0.848903
0,linear regression-1,1,req_neuroharm_mix_linr.1,4.545240,0.884066,0.884127
0,linear regression-2,2,req_neuroharm_mix_linr.2,4.626917,0.876144,0.876492
0,linear regression-3,3,req_neuroharm_mix_linr.3,4.757335,0.828342,0.828665
0,linear regression-4,4,req_neuroharm_mix_linr.4,4.686593,0.868483,0.868572


In [39]:
linr_k_frame.to_csv(output_folder + 'linr_k_frame_neuroHarmonize_harm.csv')

In [40]:
avg_linr = sep.avg_k_folds(linr_k_frame)
avg_linr

,algorithm,file_name,mae,r2,explained_variance
0,0 linear regression-0 0 linear regressio...,0 req_neuroharm_mix_linr.0 0 req_neuroha...,4.667296,0.860638,0.861352


In [41]:
linr_y_frame.to_csv(output_folder + 'linr_y_frame_neuroHarmonize_harm.csv')
linr_y_frame

,y_test,y_pred
0,27.550000,32.939004
1,52.495890,55.239775
2,31.372603,39.468910
3,50.345205,52.977468
4,77.593735,70.009146
...,...,...
272,69.424658,85.144008
273,48.730000,46.020117
274,60.183060,60.893514
275,40.355191,36.263160


In [42]:
linr_y_frame.to_csv(output_folder + 'linr_y_frame_neuroHarmonize_harm.csv')

In [43]:
linr = models[0]
linr[0]

LinearRegression()

In [44]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [45]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'neuroharm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'neuroharm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'neuroharm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'neuroharm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'neuroharm_mix_linr4.sav'))

In [46]:
llreg_k_frame, llreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('lasso regression', 'unharm_mix_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X[:,1:], y)
llreg_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,lasso regression-0,0,unharm_mix_llreg.0,4.798771,0.842336,0.845386
0,lasso regression-1,1,unharm_mix_llreg.1,4.658204,0.876017,0.876159
0,lasso regression-2,2,unharm_mix_llreg.2,4.964614,0.864690,0.866020
0,lasso regression-3,3,unharm_mix_llreg.3,5.185616,0.792915,0.793662
0,lasso regression-4,4,unharm_mix_llreg.4,5.127028,0.847420,0.847938


In [47]:
llreg_k_frame.to_csv(output_folder + 'llreg_k_frame_neuroHarmonize_harm.csv')

In [48]:
avg_llreg = sep.avg_k_folds(llreg_k_frame)
avg_llreg

,algorithm,file_name,mae,r2,explained_variance
0,0 lasso regression-0 0 lasso regression-...,0 unharm_mix_llreg.0 0 unharm_mix_llreg....,4.946847,0.844676,0.845833


In [49]:
llreg_y_frame

,y_test,y_pred
0,27.550000,39.509142
1,52.495890,55.708774
2,31.372603,44.722340
3,50.345205,55.953380
4,77.593735,66.413323
...,...,...
272,69.424658,84.523509
273,48.730000,46.762455
274,60.183060,58.132425
275,40.355191,38.743982


In [50]:
llreg_y_frame.to_csv(output_folder + 'llreg_y_frame_neuroHarmonize_harm.csv')

In [51]:
llreg = models[0]
llreg[0]

LassoLars(alpha=0.01)

In [52]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'neuroharm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'neuroharm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'neuroharm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'neuroharm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'neuroharm_mix_linr4.sav'))

In [53]:
dtree_k_frame, dtree_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('decision tree', 'unharm_mix_dtree',  tree.DecisionTreeRegressor(), ml_matrix, X[:,1:], y)
dtree_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,decision tree-0,0,unharm_mix_dtree.0,5.056562,0.856516,0.856971
0,decision tree-1,1,unharm_mix_dtree.1,5.026085,0.854762,0.854959
0,decision tree-2,2,unharm_mix_dtree.2,5.651678,0.803646,0.803651
0,decision tree-3,3,unharm_mix_dtree.3,5.436352,0.829614,0.830453
0,decision tree-4,4,unharm_mix_dtree.4,5.541807,0.807565,0.807578


In [54]:
dtree_k_frame.to_csv(output_folder + 'dtree_k_frame_neuroHarmonize_harm.csv')

In [55]:
avg_dtree = sep.avg_k_folds(dtree_k_frame)
avg_dtree

,algorithm,file_name,mae,r2,explained_variance
0,0 decision tree-0 0 decision tree-1 0 ...,0 unharm_mix_dtree.0 0 unharm_mix_dtree....,5.342497,0.830421,0.830723


In [56]:
dtree_y_frame

,y_test,y_pred
0,27.550000,33.870000
1,52.495890,62.185792
2,31.372603,28.710000
3,50.345205,48.730000
4,77.593735,60.857534
...,...,...
272,69.424658,78.517808
273,48.730000,45.660000
274,60.183060,61.972176
275,40.355191,44.280000


In [57]:
dtree_y_frame.to_csv(output_folder + 'dtree_y_frame_neuroHarmonize_harm.csv')

In [58]:
dtree = models[0]
dtree[0]

DecisionTreeRegressor()

In [65]:
regr_k_frame, regr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('MLP regression', 'unharm_mix_regr',   MLPRegressor(random_state=1, max_iter=700), ml_matrix, X[:,1:], y)
regr_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,MLP regression-0,0,unharm_mix_regr.0,4.831553,0.867962,0.868046
0,MLP regression-1,1,unharm_mix_regr.1,5.544497,0.819814,0.820567
0,MLP regression-2,2,unharm_mix_regr.2,5.747130,0.799564,0.820021
0,MLP regression-3,3,unharm_mix_regr.3,5.973852,0.769794,0.772994
0,MLP regression-4,4,unharm_mix_regr.4,5.753728,0.806759,0.809940


In [74]:
regr_k_frame.to_csv(output_folder + 'regr_k_frame_neuroHarmonize_harm.csv')

In [75]:
avg_regr = sep.avg_k_folds(regr_k_frame)
avg_regr

,algorithm,file_name,mae,r2,explained_variance
0,0 MLP regression-0 0 MLP regression-1 0 ...,0 unharm_mix_regr.0 0 unharm_mix_regr.1 ...,5.570152,0.812779,0.818313


In [76]:
regr_y_frame

,y_test,y_pred
0,27.550000,38.657202
1,52.495890,53.506360
2,31.372603,41.411756
3,50.345205,53.701124
4,77.593735,67.268063
...,...,...
272,69.424658,75.472604
273,48.730000,49.617685
274,60.183060,50.482045
275,40.355191,38.525455


In [77]:
regr_y_frame.to_csv(output_folder + 'regr_y_frame_neuroHarmonize_harm.csv')

In [78]:
regr = models[0]
regr[0]

SVR(degree=2, epsilon=0.2, kernel='poly')

In [79]:
svrp2_k_frame, svrp2_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('support vector reg poly2', 'unharm_mix_svrp2',   SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2), ml_matrix, X[:,1:], y)
svrp2_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,support vector reg poly2-0,0,unharm_mix_svrp2.0,11.516490,0.342580,0.349509
0,support vector reg poly2-1,1,unharm_mix_svrp2.1,12.120376,0.230735,0.236248
0,support vector reg poly2-2,2,unharm_mix_svrp2.2,11.407707,0.309561,0.320905
0,support vector reg poly2-3,3,unharm_mix_svrp2.3,11.844086,0.302394,0.306495
0,support vector reg poly2-4,4,unharm_mix_svrp2.4,11.696177,0.306632,0.310276


In [80]:
svrp2_k_frame.to_csv(output_folder + 'svrp2_k_frame_neuroHarmonize_harm.csv')

In [81]:
avg_svrp2 = sep.avg_k_folds(svrp2_k_frame)
avg_svrp2

,algorithm,file_name,mae,r2,explained_variance
0,0 support vector reg poly2-0 0 support v...,0 unharm_mix_svrp2.0 0 unharm_mix_svrp2....,11.716967,0.29838,0.304687


In [82]:
svrp2_y_frame

,y_test,y_pred
0,27.550000,46.593610
1,52.495890,51.089052
2,31.372603,43.661264
3,50.345205,52.626914
4,77.593735,54.702816
...,...,...
272,69.424658,62.509357
273,48.730000,48.471688
274,60.183060,48.502878
275,40.355191,52.501242


In [92]:
svrp2_y_frame.to_csv(output_folder + 'svrp2_y_frame_neuroHarmonize_harm.csv')

In [93]:
svrp2 = models[0]
svrp2[0]

ExtraTreesRegressor(random_state=0)

In [94]:
eregr_k_frame, eregr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('elasticnetCV', 'unharm_mix_eregr',  ElasticNetCV(cv=5, random_state=12), ml_matrix, X[:,1:], y)
eregr_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,elasticnetCV-0,0,unharm_mix_eregr.0,9.994718,0.482144,0.483690
0,elasticnetCV-1,1,unharm_mix_eregr.1,10.880828,0.373580,0.373622
0,elasticnetCV-2,2,unharm_mix_eregr.2,9.715762,0.479135,0.482453
0,elasticnetCV-3,3,unharm_mix_eregr.3,10.664553,0.427034,0.427035
0,elasticnetCV-4,4,unharm_mix_eregr.4,10.356312,0.439442,0.439444


In [95]:
eregr_k_frame.to_csv(output_folder + 'eregr_k_frame_neuroHarmonize_harm.csv')

In [96]:
avg_eregr = sep.avg_k_folds(eregr_k_frame)
avg_eregr

,algorithm,file_name,mae,r2,explained_variance
0,0 elasticnetCV-0 0 elasticnetCV-1 0 e...,0 unharm_mix_eregr.0 0 unharm_mix_eregr....,10.322435,0.440267,0.441249


In [97]:
eregr_y_frame

,y_test,y_pred
0,27.550000,43.509790
1,52.495890,52.724634
2,31.372603,42.667449
3,50.345205,55.938585
4,77.593735,59.821511
...,...,...
272,69.424658,68.821661
273,48.730000,49.302987
274,60.183060,49.636286
275,40.355191,53.169640


In [98]:
eregr_y_frame.to_csv(output_folder + 'eregr_y_frame_neuroHarmonize_harm.csv')

In [99]:
eregr = models[0]
eregr[0]

ElasticNetCV(cv=5, random_state=12)

In [100]:
etreg_k_frame, etreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('extra trees', 'unharm_mix_etreg',  ExtraTreesRegressor(n_estimators=100, random_state=0), ml_matrix, X[:,1:], y)
etreg_k_frame

/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_results = pd.concat([mod_results, mod_results_current_fold])
/scratch/radv/mdijsselhof/cvasl/extended_harm_paper/../cvasl/seperated.py:1055: FutureWarning: The behavior of DataFra

,algorithm,fold,file_name,mae,r2,explained_variance
0,extra trees-0,0,unharm_mix_etreg.0,3.676829,0.924363,0.924370
0,extra trees-1,1,unharm_mix_etreg.1,3.489098,0.932592,0.932754
0,extra trees-2,2,unharm_mix_etreg.2,3.506451,0.926567,0.926749
0,extra trees-3,3,unharm_mix_etreg.3,3.674335,0.922877,0.923214
0,extra trees-4,4,unharm_mix_etreg.4,3.852051,0.912325,0.912519


In [101]:
etreg_k_frame.to_csv(output_folder + 'etreg_k_frame_neuroHarmonize_harm.csv')

In [102]:
avg_etreg = sep.avg_k_folds(etreg_k_frame)
avg_etreg

,algorithm,file_name,mae,r2,explained_variance
0,0 extra trees-0 0 extra trees-1 0 ext...,0 unharm_mix_etreg.0 0 unharm_mix_etreg....,3.639753,0.923745,0.923921


In [103]:
etreg_y_frame

,y_test,y_pred
0,27.550000,37.345912
1,52.495890,55.330812
2,31.372603,36.715058
3,50.345205,50.551284
4,77.593735,65.682041
...,...,...
272,69.424658,71.272660
273,48.730000,44.720609
274,60.183060,65.800368
275,40.355191,34.828124


In [104]:
etreg_y_frame.to_csv(output_folder + 'etreg_y_frame_neuroHarmonize_harm.csv')

In [105]:
etreg = models[0]
etreg[0]

ExtraTreesRegressor(random_state=0)

In [106]:
## optional save models
#joblib.dump(etreg[0], ('../result_models/'+ 'neuroharm_mix_etreg0.sav'))
#joblib.dump(etreg[1], ('../result_models/'+ 'neuroharm_mix_etreg1.sav'))
#joblib.dump(etreg[2], ('../result_models/'+ 'neuroharm_mix_etreg2.sav'))
#joblib.dump(etreg[3], ('../result_models/'+ 'neuroharm_mix_etreg3.sav'))
#joblib.dump(etreg[4], ('../result_models/'+ 'neuroharm_mix_etreg4.sav'))

Show results ON AVERAGE for each model

In [107]:
mixed_based_neuro_harmonized_on_testmix =pd.concat([avg_linr,
                   avg_llreg,
                   avg_dtree,
                   avg_regr,
                   avg_svrp2,
                   avg_eregr,
                   avg_etreg],
                  axis=0)
mixed_based_neuro_harmonized_on_testmix

,algorithm,file_name,mae,r2,explained_variance
0,0 linear regression-0 0 linear regressio...,0 req_neuroharm_mix_linr.0 0 req_neuroha...,4.667296,0.860638,0.861352
0,0 lasso regression-0 0 lasso regression-...,0 unharm_mix_llreg.0 0 unharm_mix_llreg....,4.946847,0.844676,0.845833
0,0 decision tree-0 0 decision tree-1 0 ...,0 unharm_mix_dtree.0 0 unharm_mix_dtree....,5.342497,0.830421,0.830723
0,0 MLP regression-0 0 MLP regression-1 0 ...,0 unharm_mix_regr.0 0 unharm_mix_regr.1 ...,5.570152,0.812779,0.818313
0,0 support vector reg poly2-0 0 support v...,0 unharm_mix_svrp2.0 0 unharm_mix_svrp2....,11.716967,0.298380,0.304687
0,0 elasticnetCV-0 0 elasticnetCV-1 0 e...,0 unharm_mix_eregr.0 0 unharm_mix_eregr....,10.322435,0.440267,0.441249
0,0 extra trees-0 0 extra trees-1 0 ext...,0 unharm_mix_etreg.0 0 unharm_mix_etreg....,3.639753,0.923745,0.923921


In [108]:
mixed_based_neuro_harmonized_on_testmix.to_csv(output_folder + 'mixed_based_neuro_harmonized_on_testmix.csv')

# Running mixed model over SABRE dataset

## Here we will do an example of running  models made of the entire StrokeMRI and TOP dataset mixed as one

#### Build new models

In [109]:
TOPMRI.head(3)

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b_cbf,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf
0,sub-0668_1,50.40,1,0.617501,0.470392,0.236052,0.463182,0.819954,-0.005404,12.732741,2.821634,0.248649,0.558961,0.82831,0.719521,18.921497,87.011810,80.241341,55.807275,70.977497
1,sub-0532_1,37.02,1,0.693920,0.579498,0.294685,0.439796,0.811614,-0.006151,19.612609,2.001670,0.337168,0.862546,0.60858,0.716976,18.218930,98.415147,90.024820,66.535156,79.219991
2,sub-0529_1,30.57,1,0.641051,0.444492,0.265101,0.473141,0.796567,-0.002403,15.470687,1.955187,0.618026,0.620556,0.53582,0.596405,20.082497,106.272833,99.973628,71.505620,86.542509


In [110]:
TOPMRI.shape

(1107, 20)

In [111]:
SABRE.shape

(706, 20)

In [112]:
SABRE.columns

Index(['participant_id', 'age', 'sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf'],
      dtype='object')

In [113]:
TOPMRI.columns

Index(['participant_id', 'age', 'sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf'],
      dtype='object')

In [116]:
ml_matrix = TOPMRI.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')

In [117]:
# now we decide to do no test train split, rather take all
X_train = X
y_train = y

In [118]:
MIXlinr = LinearRegression()
MIXlinr.fit(X_train, y_train)

LinearRegression()

In [119]:
MIXllreg = linear_model.LassoLars(alpha=0.01)
MIXllreg.fit(X_train, y_train)

LassoLars(alpha=0.01)

In [120]:
MIXeregr = ElasticNetCV(cv=5, random_state=17)
MIXeregr.fit(X_train, y_train)


ElasticNetCV(cv=5, random_state=17)

In [121]:
MIXetreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
MIXetreg.fit(X_train, y_train)

ExtraTreesRegressor(random_state=0)

In [122]:
##  Save these four best models

In [123]:
## optional save models
#joblib.dump(MIXlinr, ('../result_models/'+  'neuro_harm_mix_MIXlinr.sav'))
#joblib.dump(MIXllreg, ('../result_models/'+ 'neuro_harmm_mix_MIXllreg.sav'))
#joblib.dump(MIXeregr, ('../result_models/'+ 'neuro_harm_mix_MIXeregr.sav'))
#joblib.dump(MIXetreg, ('../result_models/'+ 'neuro_harm_mix_MIXetreg.sav'))

In [124]:
sabre_ml_matrix = SABRE.drop('participant_id', axis=1)
X_sabre = sabre_ml_matrix.drop('age', axis =1)
X_sabre = X_sabre.values
X_sabre = X_sabre.astype('float')
y_sabre = sabre_ml_matrix['age'].values
y_sabre=y_sabre.astype('float')

In [125]:
X_sabre_test = X_sabre
y_sabre_test = y_sabre

In [126]:
y_sabre_pred = MIXlinr.predict(X_sabre_test)

In [127]:
data= [[
    'Linear Reg',
    'unharm_mix_linr0.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXlinr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
linr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_sabre

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,unharm_mix_linr0.sav,22.514418,-24.956691,-22.40761


In [128]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_linr_sabre = pd.DataFrame(data)
y_frame_linr_sabre

,real,predicted
0,79.0,88.164582
1,70.0,45.754473
2,62.0,42.456166
3,78.0,60.485689
4,68.0,160.293791
...,...,...
701,66.0,30.745296
702,72.0,33.581636
703,73.0,75.167885
704,65.0,27.523750


In [133]:
y_frame_linr_sabre.to_csv(output_folder + 'y_frame_linr_sabre_neuroHarmonize_harm.csv')

In [134]:
y_sabre_pred = MIXllreg.predict(X_sabre_test)

In [135]:
data= [[
    'Lasso',
    'unharm_mix_lassor0.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXllreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
llreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_sabre

,algorithm,file_name,mae,r2,explained_variance
0,Lasso,unharm_mix_lassor0.sav,27.373801,-37.546088,-34.167996


In [136]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_llreg_sabre = pd.DataFrame(data)
y_frame_llreg_sabre

,real,predicted
0,79.0,88.988009
1,70.0,40.940142
2,62.0,54.129869
3,78.0,56.166283
4,68.0,190.775716
...,...,...
701,66.0,22.373161
702,72.0,26.987787
703,73.0,78.833793
704,65.0,18.231910


In [137]:
y_frame_llreg_sabre.to_csv(output_folder + 'y_frame_llreg_sabre_neuroHarmonize_harm.csv')

In [138]:
y_sabre_pred = MIXeregr.predict(X_sabre_test)

In [139]:
data= [[
    'ElasticnetCV',
    'harm_mix_elasticregr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXeregr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
eregr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_sabre

,algorithm,file_name,mae,r2,explained_variance
0,ElasticnetCV,harm_mix_elasticregr.sav,14.756667,-6.022811,-1.469568


In [140]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_eregr_sabre = pd.DataFrame(data)
y_frame_eregr_sabre

,real,predicted
0,79.0,58.333517
1,70.0,61.426284
2,62.0,63.550552
3,78.0,49.977901
4,68.0,58.869822
...,...,...
701,66.0,58.851974
702,72.0,48.289359
703,73.0,49.190872
704,65.0,57.221241


In [141]:
y_frame_eregr_sabre.to_csv(output_folder + 'y_frame_eregr_sabre_neuroHarmonize_harm.csv')

In [142]:
y_sabre_pred = MIXetreg.predict(X_sabre_test)

In [143]:
data= [[
    'Extra trees',
    'harm_mix_etreg.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXetreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
etreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_sabre

,algorithm,file_name,mae,r2,explained_variance
0,Extra trees,harm_mix_etreg.sav,7.314053,-0.979878,-0.157382


In [144]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_etregr_sabre = pd.DataFrame(data)
y_frame_etregr_sabre

,real,predicted
0,79.0,74.377571
1,70.0,66.181205
2,62.0,61.205921
3,78.0,67.687562
4,68.0,70.054308
...,...,...
701,66.0,50.033954
702,72.0,57.025023
703,73.0,71.878381
704,65.0,50.933096


In [145]:
y_frame_etregr_sabre.to_csv(output_folder + 'y_frame_etregr_sabre_neuroHarmonize_harm.csv')

In [146]:
mix_based_neuroHarmonize_on_sabre =pd.concat([linr_results_sabre,
                   llreg_results_sabre,
                   eregr_results_sabre,
                  etreg_results_sabre],
                  axis=0)
mix_based_neuroHarmonize_on_sabre

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,unharm_mix_linr0.sav,22.514418,-24.956691,-22.407610
0,Lasso,unharm_mix_lassor0.sav,27.373801,-37.546088,-34.167996
0,ElasticnetCV,harm_mix_elasticregr.sav,14.756667,-6.022811,-1.469568
0,Extra trees,harm_mix_etreg.sav,7.314053,-0.979878,-0.157382


In [148]:
mix_based_neuroHarmonize_on_sabre.to_csv(output_folder + 'mix_based_neuroHarmonize_on_sabre.csv')

# Running mixed model over Insight46 dataset

## Here we will do an example of running allthe [0] models

In [149]:
insight_ml_matrix = Insight46.drop('participant_id', axis=1)
X_insight = insight_ml_matrix.drop('age', axis =1)
X_insight = X_insight.values
X_insight = X_insight.astype('float')
y_insight = insight_ml_matrix['age'].values
y_insight= y_insight.astype('float')

In [150]:
X_insight_test = X_insight
y_insight_test = y_insight

In [151]:
y_insight_pred = MIXlinr.predict(X_insight_test)

In [152]:
data= [[
    'Linear Reg',
    'neuroharm_mix_linr0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXlinr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
linr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_insight

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,neuroharm_mix_linr0.sav,58.477496,-3312.227066,-1139.526215


In [153]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_linr_insight = pd.DataFrame(data)
y_frame_linr_insight

,real,predicted
0,71.072222,-3.931089
1,73.000000,45.302896
2,74.000000,-8.851411
3,71.644444,-5.386275
4,71.694444,-8.208094
...,...,...
612,72.000000,35.835459
613,73.000000,137.099005
614,72.000000,34.320204
615,70.605556,-10.147587


In [154]:
y_frame_linr_insight.to_csv(output_folder + 'y_frame_linr_insight_neuroHarmonize_harm.csv')

In [155]:
y_insight_pred = MIXllreg.predict(X_insight_test)

In [156]:
data= [[
    'Lasso',
    'neuroharm_mix_lassor0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXllreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
llreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_insight

,algorithm,file_name,mae,r2,explained_variance
0,Lasso,neuroharm_mix_lassor0.sav,71.577066,-4920.225303,-1656.44171


In [157]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_llreg_insight = pd.DataFrame(data)
y_frame_llreg_insight

,real,predicted
0,71.072222,-25.522229
1,73.000000,26.007242
2,74.000000,-26.268920
3,71.644444,-22.719435
4,71.694444,-27.485713
...,...,...
612,72.000000,23.087430
613,73.000000,150.023835
614,72.000000,24.925866
615,70.605556,-26.021273


In [158]:
y_frame_llreg_insight.to_csv(output_folder + 'y_frame_llreg_insight_neuroHarmonize_harm.csv')

In [159]:
y_insight_pred = MIXeregr.predict(X_insight_test)

In [160]:
data= [[
    'ElasticnetCV',
    'neuroharm_mix_elasticregr.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXeregr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
eregr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_insight

,algorithm,file_name,mae,r2,explained_variance
0,ElasticnetCV,neuroharm_mix_elasticregr.sav,17.262668,-242.285721,-43.913813


In [161]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_eregr_insight = pd.DataFrame(data)
y_frame_eregr_insight

,real,predicted
0,71.072222,51.742313
1,73.000000,54.897203
2,74.000000,51.124073
3,71.644444,52.664894
4,71.694444,47.600143
...,...,...
612,72.000000,69.055457
613,73.000000,78.234798
614,72.000000,71.114700
615,70.605556,56.625540


In [162]:
y_frame_eregr_insight.to_csv(output_folder + 'y_frame_eregr_insight_neuroHarmonize_harm.csv')

In [163]:
y_insight_pred = MIXetreg.predict(X_insight_test)

In [164]:
data= [[
    'Extra trees',
    'neuroharm_mix_etreg.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXetreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
etreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_insight

,algorithm,file_name,mae,r2,explained_variance
0,Extra trees,neuroharm_mix_etreg.sav,12.544772,-155.137282,-51.059374


In [165]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_etreg_insight = pd.DataFrame(data)
y_frame_etreg_insight

,real,predicted
0,71.072222,67.659487
1,73.000000,69.586604
2,74.000000,57.743520
3,71.644444,56.141074
4,71.694444,58.799660
...,...,...
612,72.000000,69.330714
613,73.000000,81.089387
614,72.000000,72.636991
615,70.605556,45.561784


In [166]:
y_frame_etreg_insight.to_csv(output_folder + 'y_frame_etreg_insight_neuroHarmonize_harm.csv')

In [167]:
mix_based_neuroHarmonize_on_insight =pd.concat([linr_results_insight,
                   llreg_results_insight,
                   eregr_results_insight,
                   etreg_results_insight],
                  axis=0)
mix_based_neuroHarmonize_on_insight

,algorithm,file_name,mae,r2,explained_variance
0,Linear Reg,neuroharm_mix_linr0.sav,58.477496,-3312.227066,-1139.526215
0,Lasso,neuroharm_mix_lassor0.sav,71.577066,-4920.225303,-1656.441710
0,ElasticnetCV,neuroharm_mix_elasticregr.sav,17.262668,-242.285721,-43.913813
0,Extra trees,neuroharm_mix_etreg.sav,12.544772,-155.137282,-51.059374


In [168]:
mix_based_neuroHarmonize_on_insight.to_csv(output_folder + 'mix_based_neuroHarmonize_on_insight.csv')